In [ ]:
from datetime import datetime

import matplotlib.pyplot as plt

from turtle_quant_1.data_processing.processor import DataProcessor
from turtle_quant_1.strategies.candlesticks import (
    EngulfingPattern,
    MomentumPattern,
    MultiplePattern,
)

## Data pre-processing

In [ ]:
symbol = "XOM"

In [ ]:
dates = {"start": datetime(2024, 1, 1), "end": datetime(2025, 7, 31)}

data_processor = DataProcessor()
data = data_processor.load_data(
    symbol=symbol,
    start_date=dates["start"],
    end_date=dates["end"],
    impute_data=True,
)

## Select strategies to test

In [ ]:
strategies = [
    EngulfingPattern(),
    MomentumPattern(),
    MultiplePattern(),
]

## Visualization

In [ ]:
import plotly.graph_objects as go
from IPython.display import display
from plotly.subplots import make_subplots

for strategy in strategies:
    scores = strategy.generate_historical_scores(data=data, symbol=symbol)

    fig = make_subplots(
        rows=2,
        cols=1,
        shared_xaxes=True,
        vertical_spacing=0.1,
        row_heights=[0.6, 0.4],
        subplot_titles=("Close Price", f"{type(strategy).__name__} Score"),
    )

    # Price plot
    fig.add_trace(
        go.Scatter(
            x=data["datetime"],
            y=data["Close"],
            mode="lines",
            name="Close Price",
            line=dict(color="blue"),
        ),
        row=1,
        col=1,
    )

    # Score plot
    fig.add_trace(
        go.Scatter(
            x=data["datetime"],
            y=scores,
            mode="lines",
            name="Normalized Score",
            line=dict(color="orange"),
        ),
        row=2,
        col=1,
    )

    # Add horizontal threshold lines
    thresholds = [
        {"y": 0.0, "color": "black", "dash": "dash", "name": "Zero Line"},
        {"y": 0.3, "color": "green", "dash": "dash", "name": "Buy Threshold"},
        {"y": -0.3, "color": "red", "dash": "dash", "name": "Sell Threshold"},
    ]
    for threshold in thresholds:
        fig.add_hline(
            y=threshold["y"],
            line=dict(color=threshold["color"], dash=threshold["dash"], width=1),
            row=2,
            col=1,
            annotation_text=threshold["name"],
            annotation_position="top left",
        )

    # Layout
    fig.update_layout(
        height=600,
        width=1000,
        title_text=f"{type(strategy).__name__} - {symbol} Analysis",
        legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1),
    )

    fig.update_yaxes(title_text="Price", row=1, col=1)
    fig.update_yaxes(title_text="Score", row=2, col=1)
    fig.update_xaxes(title_text="Date", row=2, col=1)

    display(fig)